## Train error vs Test error

Illustration of how the performance of an estimator on unseen data (test data)
is not the same as the performance on training data. As the regularization
increases the performance on train decreases while the performance on test
is optimal within a range of values of the regularization parameter.
The example with an Elastic-Net regression model and the performance is
measured using the explained variance a.k.a. R^2.

Author: Alexandre Gramfort <alexandre.gramfort@inria.fr>

License: BSD 3 clause

Source : https://github.com/scikit-learn/scikit-learn/blob/master/examples/model_selection/plot_train_error_vs_test_error.py

*Import packages*

In [1]:
import numpy as np
from sklearn import linear_model

*Import polyaxon package*

In [ ]:
from polyaxon_client.tracking import Experiment

*Generate sample data*

In [3]:
n_samples_train, n_samples_test, n_features = 75, 150, 500
np.random.seed(0)
coef = np.random.randn(n_features)
coef[50:] = 0.0  # only the top 10 features are impacting the model
X = np.random.randn(n_samples_train + n_samples_test, n_features)
y = np.dot(X, coef)

*Split train and test data*

In [4]:
X_train, X_test = X[:n_samples_train], X[n_samples_train:]
y_train, y_test = y[:n_samples_train], y[n_samples_train:]

*Compute train and test errors*

In [ ]:
alphas = np.logspace(-5, 1, 60)

enet = linear_model.ElasticNet(l1_ratio=0.7, max_iter=10000)
train_errors = list()
test_errors = list()
for alpha in alphas:
    experiment = Experiment()
    experiment.create()
    enet.set_params(alpha=alpha)
    enet.fit(X_train, y_train)
    train_errors.append(enet.score(X_train, y_train))
    test_errors.append(enet.score(X_test, y_test))
    #Track parameters and metrics, the result can me exploited in the UI.
    experiment.log_metrics(train_errors = enet.score(X_train, y_train))
    experiment.log_metrics(test_errors = enet.score(X_test, y_test))
    experiment.log_params(l1_ratio = 0.7, max_iter = 10000, alpha = alpha)

i_alpha_optim = np.argmax(test_errors)
alpha_optim = alphas[i_alpha_optim]
print("Optimal regularization parameter : %s" % alpha_optim)

*Estimate the coef_ on full data with optimal regularization parameter*

In [ ]:
enet.set_params(alpha=alpha_optim)
coef_ = enet.fit(X, y).coef_

*Plot results functions*

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(2, 1, 1)
plt.semilogx(alphas, train_errors, label='Train')
plt.semilogx(alphas, test_errors, label='Test')
plt.vlines(alpha_optim, plt.ylim()[0], np.max(test_errors), color='k',
           linewidth=3, label='Optimum on test')
plt.legend(loc='lower left')
plt.ylim([0, 1.2])
plt.xlabel('Regularization parameter')
plt.ylabel('Performance')

*Show estimated coef_ vs true coef*

In [ ]:
plt.subplot(2, 1, 2)
plt.plot(coef, label='True coef')
plt.plot(coef_, label='Estimated coef')
plt.legend()
plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.26)
plt.show()